In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))

import requests
import dotenv

dotenv.load_dotenv()

True

In [2]:
def get_first_oauth_token():
    """
    Obtém token OAuth do MercadoLibre usando authorization code
    """
    
    # URL do endpoint
    url = "https://api.mercadolibre.com/oauth/token"
    
    # Headers
    headers = {
        'accept': 'application/json',
        'content-type': 'application/x-www-form-urlencoded'
    }
    
    # Dados do formulário
    data = {
        'grant_type': 'authorization_code',
        'client_id': os.getenv('MELI_APP_ID'),  # ou substitua pela sua APP_ID
        'client_secret': os.getenv('MELI_SECRET_KEY'),  # ou substitua pela sua SECRET_KEY
        'code': os.getenv('MELI_CODE'),  # código de autorização
        'redirect_uri': os.getenv('MELI_REDIRECT_URI'),  # URI de redirecionamento
        #'code_verifier': os.getenv('MELI_CODE_VERIFIER')  # verificador PKCE
    }
    
    try:
        # Fazer a requisição POST
        response = requests.post(url, headers=headers, data=data)
        
        # Verificar se a requisição foi bem-sucedida
        response.raise_for_status()
        
        # Retornar a resposta JSON
        return response.json()
        
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição: {e}")
        return None
    

def get_oauth_token():
    """
    Obtém token OAuth do MercadoLibre usando refresh token
    """
    
    # URL do endpoint
    url = "https://api.mercadolibre.com/oauth/token"

    # Headers
    headers = {
        'accept': 'application/json',
        'content-type': 'application/x-www-form-urlencoded'
    }

    # Dados do formulário
    data = {
        'grant_type': 'refresh_token',
        'client_id': os.getenv('MELI_APP_ID'),
        'client_secret': os.getenv('MELI_SECRET_KEY'),
        'refresh_token': os.getenv('MELI_REFRESH_TOKEN')
    }

    try:
        # Fazer a requisição POST
        response = requests.post(url, headers=headers, data=data)

        # Verificar se a requisição foi bem-sucedida
        response.raise_for_status()

        # Retornar a resposta JSON
        return response.json()

    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição: {e}")
        return None

In [3]:
def update_env_file(response):
    """
    Atualiza o arquivo .env com o access_token e refresh_token obtidos
    """
    if response and 'access_token' in response and 'refresh_token' in response:
        # Ler o conteúdo atual do .env
        env_path = '../.env'
        env_vars = {}
        
        # Se o arquivo existe, ler as variáveis existentes
        if os.path.exists(env_path):
            with open(env_path, 'r') as file:
                for line in file:
                    line = line.strip()
                    if line and '=' in line and not line.startswith('#'):
                        key, value = line.split('=', 1)
                        env_vars[key] = value
        
        # Atualizar com os novos tokens
        env_vars['MELI_ACCESS_TOKEN'] = response['access_token']
        env_vars['MELI_REFRESH_TOKEN'] = response['refresh_token']
        
        # Escrever de volta no arquivo
        with open(env_path, 'w') as file:
            for key, value in env_vars.items():
                if key in ['MELI_ACCESS_TOKEN', 'MELI_REFRESH_TOKEN']:
                    file.write(f'{key}="{value}"\n')
                else:
                    file.write(f'{key}={value}\n')

        print("Tokens atualizados no arquivo .env com sucesso!")
        print(f"Access Token: {response['access_token'][:20]}...")
        print(f"Refresh Token: {response['refresh_token'][:20]}...")
    else:
        print("Erro: Response não contém os tokens necessários")

In [4]:
response = get_oauth_token()
if response:
    print("Token OAuth obtido com sucesso:")
    #print(response)
    update_env_file(response)

dotenv.load_dotenv()

Token OAuth obtido com sucesso:
Tokens atualizados no arquivo .env com sucesso!
Access Token: APP_USR-876268789520...
Refresh Token: TG-68ffdd5d6f00cf000...


True

In [ ]:
! curl -H 'Authorization: Bearer ' \
https://api.mercadolibre.com/users/me

{"id":817670010,"nickname":"LUCASEDMUNDOMSILVA","registration_date":"2022-06-14T08:08:45.000-04:00","first_name":"Lucas Edmundo","last_name":"Mello Silva","gender":"","country_id":"BR","email":"lucasedmundo11@gmail.com","identification":{"number":"51331931819","type":"CPF"},"address":{"address":null,"city":null,"state":null,"zip_code":null},"phone":{"area_code":null,"extension":"","number":null,"verified":false},"alternative_phone":{"area_code":"","extension":"","number":""},"user_type":"normal","tags":["credits_profile","normal","messages_as_seller"],"logo":null,"points":2,"site_id":"MLB","permalink":"http://perfil.mercadolivre.com.br/LUCASEDMUNDOMSILVA","seller_experience":"NEWBIE","bill_data":{"accept_credit_note":null},"seller_reputation":{"level_id":null,"power_seller_status":null,"transactions":{"canceled":0,"completed":0,"period":"historic","ratings":{"negative":0,"neutral":0,"positive":0},"total":0},"metrics":{"sales":{"period":"365 days","completed":0},"claims":{"period":"365 

In [ ]:
url = f"https://api.mercadolibre.com/sites/MLA/search"
params = {
    'nickname': "iPhone 15",
    'limit': 1,
    'offset': 50
}
headers = {
    'Authorization': f"Bearer {os.getenv('MELI_ACCESS_TOKEN')}"
}

session = requests.Session()
response = session.get(url, data=params, headers=headers, timeout=30)
#response.raise_for_status()

#data = response.json()
results = data.get('results', [])
data

In [ ]:
! curl -X POST -H 'Authorization: Bearer ' -H "Content-type: application/json" -d '{"site_id":"MLA"}' 'https://api.mercadolibre.com/users/test_user'

In [ ]:
!curl -k -X GET -H 'Authorization: Bearer ' https://api.mercadolibre.com/items/MLB5286384022

{"message":"Access to the requested resource is forbidden","error":"access_denied","status":403,"cause":null}

In [ ]:
!curl GET -X -H 'Authorization: Bearer '  https://api.mercadolibre.com/users/{817670010}

curl: (6) Could not resolve host: GET
curl: (3) URL rejected: Malformed input to a URL function
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<HTML><HEAD><META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=iso-8859-1">
<TITLE>ERROR: The request could not be satisfied</TITLE>
</HEAD><BODY>
<H1>403 ERROR</H1>
<H2>The request could not be satisfied.</H2>
<HR noshade size="1px">
This distribution is not configured to allow the HTTP request method that was used for this request. The distribution supports only cachable requests.
We can't connect to the server for this app or website at this time. There might be too much traffic or a configuration error. Try again later, or contact the app or website owner.
<BR clear="all">
If you provide content to customers through CloudFront, you can find steps to troubleshoot and help prevent this error by reviewing the CloudFront documentation.
<BR clear="all">
<HR noshade size="1px">
<PRE>